<a href="https://colab.research.google.com/github/euphoria96/KB_Smishing_dacon/blob/master/14%ED%9A%8C_%EB%8C%80%ED%9A%8C_%EC%98%88%EC%B8%A1_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dacon 14회 KB 금융문자 분석 모델링 경진대회
### euphoria
### 2020년 1월 17일

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab

/content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

## 1. 필요한 패키지 불러오기

In [0]:
# data preprocessing
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import pickle
# text tokenizing
import re
from konlpy.tag import Mecab
# modeling
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, SpatialDropout1D, Dropout
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.metrics import roc_auc_score
tf.logging.set_verbosity(tf.logging.ERROR)

## 2. get_prediction
1. 이 함수는 테스트에 해당하는 csv 파일 경로를 매개변수로 가집니다.
2. 이 함수는 미리 만들어진 모델을 불러와 예측을 수행합니다.
3. 이 함수는 예측치를 불러온 csv 파일에 넣습니다. 이 때 예측치에 해당하는 column name은 smishing 입니다.
4. 이 함수는 데이터프레임을 리턴합니다.

In [0]:
def text_preprocessing(text_list):
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Mecab()
    token_list, ngram2_list = [], []
    for text in text_list:
        txt = re.sub('[^가-힣a-z]',' ',text.lower())
        txt = re.sub('x{1,}',' ',txt)
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords or type(t)!= float]
        token_list.append(' '.join(token))
        ngram2 = [token[i]+'.'+token[i+1] for i in range(len(token)-1)]
        ngram2_list.append(' '.join(ngram2))
    return token_list, ngram2_list

def text2sequence_test(tokenizer, test_text, max_len=1000):
    test_seq = tokenizer.texts_to_sequences(test_text)
    X_test = pad_sequences(test_seq, maxlen=max_len)
    return X_test

auc_ = 0
def auc_score(y_true, y_pred):
    global auc_
    try:
        auc_ = roc_auc_score( y_true, y_pred, average='macro', sample_weight = None).astype('float32')
    except ValueError:
        pass
    return auc_

def auc(y_true, y_pred):
    score = tf.py_func( lambda y_true, y_pred : auc_score(y_true, y_pred) , [y_true, y_pred], 'float32', stateful = False, name = 'sklearnAUC' )
    return score

In [0]:
def get_prediction(test_file_path):
    '''
    Args: String
    Return: Pandas DataFrame    
    '''
    
    '''1. 테스트 파일 불러오기'''
    test = pd.read_csv(test_file_path)
    
    '''2. 모델 불러오기'''
    with open('1_Model/tokenizer_01-16 07:13:11.pickle', 'rb') as f:
        tokenizer_test = pickle.load(f)

    with open('1_Model/01-16 09:45:06_BiLSTM_bigram_660.json', 'r') as ff:
        json_model = ff.read()
    model_test = model_from_json(json_model)
    model_test.load_weights('1_Model/01-16 09:45:06_BiLSTM_bigram_660.h5')
    
    '''3. 예측 전 필요한 사항 진행하기'''
    test['token_txt'], test['bigram'] = text_preprocessing(test.text)
    test_X = text2sequence_test(tokenizer_test, test['token_txt'], max_len=660)
    test_X2 = text2sequence_test(tokenizer_test, test['bigram'], max_len=660)
    
    model_test.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc])

    '''4. 예측 진행하기'''
    y_pred = model_test.predict(test_X2, batch_size=128)
    
    '''5. 예측치 데이터프레임에 합치기'''
    test['smishing'] = y_pred
    submission = test[['id','smishing']]
    #submission.to_csv('submission.csv',index=False)

    return submission

In [0]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/smishing/')
y_pred = get_prediction('0_Data/public_test.csv')